In [ ]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

MODEL_NAME = os.getenv("OPENAI_MODEL", "gpt-4.1-mini")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ["API_KEY"])

# Tavily APIクライアントを生成
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ["API_KEY"].strip())

# Tavily APIクライアントを生成
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"].strip())

# モデル名
MODEL_NAME = "gpt-4o-mini"


# メッセージを格納するリスト
messages=[]

# キャラクター設定
SYSTEM_PROMPT = "あなたは猫のキャラクターとして振る舞うチャットボットです。語尾に「にゃ」を付け、明るく親しみやすく返答してください。"

# システムプロンプトを先頭に追加（キャラクターが消えないように固定）
messages.append({"role": "system", "content": SYSTEM_PROMPT})


# ツール定義
def define_tools():
    return [
        {
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "ユーザーが検索を求めた場合にWeb検索を行う",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "検索したい質問文"},
                    },
                    "required": ["question"],
                },
            },
        }
    ]


# 検索結果を返す関数の作成
def get_search_result(question):
    try:
        response = tavily_client.search(question)
        return json.dumps({"result": response.get("results", [])}, ensure_ascii=False)
    except Exception as e:
        # ★ 失敗理由を確認できるように一時的に表示
        print("Tavily error:", repr(e))
        return json.dumps({"result": [], "error": str(e)}, ensure_ascii=False)


tools = define_tools()


def ask_with_tools(messages):
    return client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )


def handle_tool_calls(first_response):
    msg = first_response.choices[0].message
    tool_calls = msg.tool_calls or []

    messages.append(msg)

    for tool in tool_calls:
        function_name = tool.function.name
        arguments = json.loads(tool.function.arguments or "{}")

        if function_name == "get_search_result":
            function_response = get_search_result(**arguments)
        else:
            function_response = json.dumps({"error": f"未対応の関数です: {function_name}"}, ensure_ascii=False)

        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool.id,
                "content": function_response,
            }
        )

    response_after = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

    return (response_after.choices[0].message.content or "").strip()


def trim_messages():
    # systemを除いた直近8件の会話履歴のみを保持する。超過分はsystemの次から削除する
    if len(messages) > 9:
        messages.pop(1)


try:
    while(True):
        # ユーザーからの質問を受付
        message = input("メッセージを入力:")
        # 質問が入力されなければ終了
        if message.strip()=="":
            break
        print(f"質問:{message}")

        # メッセージにユーザーからの質問を追加
        messages.append({"role": "user", "content": message.strip()})

        trim_messages()

        # APIへリクエスト
        first = ask_with_tools(messages)

        # tool_calls が発生した場合
        if first.choices[0].finish_reason == "tool_calls":
            response_message = handle_tool_calls(first)
        else:
            response_message = (first.choices[0].message.content or "").strip()

        # 言語モデルからの回答を表示
        print(response_message)

        # メッセージに言語モデルからの回答を追加
        messages.append({"role": "assistant", "content": response_message})

        trim_messages()

except KeyboardInterrupt:
    print("処理を中断しました。")

print("---ご利用ありがとうございました！---")